In [8]:
import numpy as np
from datetime import datetime, timedelta
from quantbot.core.data import BarData
from quantbot.engine import QuantBotEngine
from quantbot.factors.technical import SMA, EMA, RSI, MACD, BollingerBands, ATR, STOCH, OBV


def generate_sample_data(num_bars: int = 100) -> list:
    """
    生成示例数据
    
    Args:
        num_bars: 生成的Bar数量
        
    Returns:
        BarData列表
    """
    bars = []
    base_time = datetime.now()
    base_price = 100.0
    
    for i in range(num_bars):
        # 生成随机价格变化
        price_change = np.random.normal(0, 1) * 0.5
        base_price += price_change
        
        # 确保价格为正
        base_price = max(base_price, 10.0)
        
        # 生成OHLC数据
        high = base_price + abs(np.random.normal(0, 0.5))
        low = base_price - abs(np.random.normal(0, 0.5))
        open_price = base_price + np.random.normal(0, 0.2)
        close = base_price
        
        # 确保OHLC逻辑正确
        high = max(high, open_price, close)
        low = min(low, open_price, close)
        
        volume = np.random.uniform(1000, 10000)
        amount = volume * close
        
        bar = BarData(
            timestamp=base_time + timedelta(minutes=i),
            open_price=open_price,
            high=high,
            low=low,
            close=close,
            volume=volume,
            amount=amount
        )
        
        bars.append(bar)
    
    return bars



In [10]:
"""主函数"""
print("QuantBot框架示例 - 增强版QuantBotEngine")
print("=" * 60)
buffer_size = 60

# 1. 创建QuantBot引擎
print("1. 创建QuantBot引擎...")
engine = QuantBotEngine(buffer_size=buffer_size)
print(f"   引擎状态: {engine}")
print(f"   初始健康状态: {engine.get_health_status()['status']}")

# 2. 注册技术指标因子
print("2. 注册技术指标因子...")

try:
    # 基础移动平均线
    sma_5 = SMA(5, 'sma_5_close')
    sma_20 = SMA(20, 'sma_20_close')
    ema_12 = EMA(12, 'ema_12_close')
    ema_26 = EMA(26, 'ema_26_close')
    
    # RSI指标
    rsi_14 = RSI(14, 'rsi_14_close')
    
    # MACD指标系列 (合并为单个因子，输出macd, signal, histogram)
    macd = MACD(12, 26, 9, 'macd_12_26_9_close')
    
    # 布林带指标（多输出因子）
    bb = BollingerBands(20, 2.0, 'bb_20_2')
    
    # 新增技术指标
    atr_14 = ATR(14, 'atr_14')
    stoch = STOCH(14, 3, 3, 'stoch_14_3_3')
    obv = OBV('obv')
    
    # 注册所有因子
    factors = [
        sma_5, sma_20, ema_12, ema_26, rsi_14,
        macd, bb,
        atr_14, stoch, obv
    ]
    
    for factor in factors:
        engine.register_factor(factor)
        print(f"   注册因子: {factor.name}，output_names: {factor.output_names}")
    
    print(f"   成功注册 {len(factors)} 个因子")
    print(f"   实际因子输出数量: {len(engine.get_registered_factors())} 个")
    print(f"   MACD(3个输出) + 布林带(3个输出) + STOCH(2个输出) + 其他单输出因子")
    
except Exception as e:
    print(f"   因子注册失败: {e}")
    raise e

# 3. 检查依赖关系和引擎状态
print("3. 分析因子依赖关系...")
calculation_order = engine.get_calculation_order()
print(f"   计算顺序: {calculation_order}...（显示前5个）")

dependency_info = engine.get_dependency_info()
print("   主要依赖关系:")
for factor_name, deps in list(dependency_info.items()):
    if deps:
        print(f"     {factor_name} <- {deps}")

# 检查循环依赖
if engine.has_circular_dependency():
    print("   ⚠️  警告: 检测到循环依赖!")
else:
    print("   ✓ 依赖关系正常，无循环依赖")

# 4. 生成和添加数据
print("\n4. 生成示例数据并开始计算...")
sample_data = generate_sample_data(100)  # 增加数据量以确保所有指标都有输出

print("   开始逐个添加数据并计算因子...")
results_history = []

# for i, bar in enumerate(sample_data):
#     try:
#         # 添加Bar数据
#         engine.add_bar(bar)
        
#         # 计算所有因子
#         results = engine.calculate_factors()
        
#         results_history.append(results.copy())
        
#         # 每15个Bar打印一次结果
#         if (i + 1) % 15 == 0:
#             print(f"\n   第 {i + 1} 个Bar的因子计算结果:")
#             print(f"     时间: {bar.timestamp.strftime('%H:%M:%S')}")
#             print(f"     收盘价: {bar.close:.2f}")
            
#             # 显示主要指标
#             key_indicators = ['sma_5_close', 'ema_12_close', 'rsi_14_close', 'bb_20_2', 'atr_14', 'obv']
#             for name in key_indicators:
#                 value = results.get(name)
#                 print(f" {name}: {value}")
                    
#     except Exception as e:
#         print(f"   第 {i + 1} 个Bar处理失败: {e}")
#     continue

QuantBot框架示例 - 增强版QuantBotEngine
1. 创建QuantBot引擎...
   引擎状态: QuantBotEngine(factors=0, buffer_size=60, health=healthy)
   初始健康状态: healthy
2. 注册技术指标因子...
   注册因子: sma_5_close，output_names: ['default']
   注册因子: sma_20_close，output_names: ['default']
   注册因子: ema_12_close，output_names: ['default']
   注册因子: ema_26_close，output_names: ['default']
   注册因子: rsi_14_close，output_names: ['default']
   注册因子: macd_12_26_9_close，output_names: ['macd', 'signal', 'histogram']
   注册因子: bb_20_2，output_names: ['upper', 'middle', 'lower']
   注册因子: atr_14，output_names: ['default']
   注册因子: stoch_14_3_3，output_names: ['k', 'd']
   注册因子: obv，output_names: ['default']
   成功注册 10 个因子
   实际因子输出数量: 15 个
   MACD(3个输出) + 布林带(3个输出) + STOCH(2个输出) + 其他单输出因子
3. 分析因子依赖关系...
   计算顺序: ['sma_5_close', 'sma_20_close', 'ema_12_close', 'ema_26_close', 'rsi_14_close', 'bb_20_2_upper', 'bb_20_2_middle', 'bb_20_2_lower', 'atr_14', 'stoch_14_3_3_k', 'stoch_14_3_3_d', 'obv', 'macd_12_26_9_close_macd', 'macd_12_26_9_close_signal'